<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preamble" data-toc-modified-id="Preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preamble</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Classes" data-toc-modified-id="Classes-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Classes</a></span></li></ul></li><li><span><a href="#Main" data-toc-modified-id="Main-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Main</a></span></li></ul></div>

# Preamble

## Imports

In [1]:
import pandas as pd
import numpy as np
import time
import sys
sys.setrecursionlimit(10**6)

## Classes

In [2]:
class ConnectionEngine():
    def __init__(self,num_people=None,num_connections=None):
        self.num_people = num_people
        self.num_connections = num_connections
        
    def _build_connection_list(agent,population,num_connections,_cnt=0):
        # Break counter
        if _cnt == 0:
            _cnt += 1
        print(_cnt)
        # Return IDs of people with connections less than num_connections
        available_to_connect = (
            lambda agent,population: population.drop(agent).query('num_connections < {}'
                                                                     .format(num_connections)
                                                                    ).index
        )

        # Update number of connections
        population['num_connections'] = population.connections.apply(len)
        # Get other agents available to connect
        available = available_to_connect(agent,population)
        # Randomly choose connection
        if len(available) > 0:
            connection = np.random.choice(available)
            # Make connection
            population.iloc[connection].connections.append(agent)
            population.iloc[agent].connections.append(connection)

        # Update number of connections
        population['num_connections'] = population.connections.apply(len)
        if _cnt < 10:
            while population.num_connections[agent] < num_connections:
                _build_connection_list(agent,
                                       population,
                                       num_connections,
                                       _cnt=_cnt)

        return population
    
    def create_connections(self):
        num_connections = self.num_connections
        num_people = self.num_people
        population = pd.DataFrame(
            {
                'index': [i for i in range(0,num_people)],
                'connections': [[] for i in range(0,num_people)],
            }
        )
        
        for _per in population.index:
            _build_connection_list(_per,population,num_connections)
            
        self.population = population
        
        return population
            


# Main

For some reason, only works with function outside of class

In [3]:
def _build_connection_list(agent,population,num_connections,_cnt=0):
    # Break counter
    if _cnt == 0:
        _cnt += 1
    # Return IDs of people with connections less than num_connections
    available_to_connect = (
        lambda agent,population: population.drop(agent).query('num_connections < {}'
                                                                 .format(num_connections)
                                                                ).index
    )

    # Update number of connections
    population['num_connections'] = population.connections.apply(len)
    # Get other agents available to connect
    available = available_to_connect(agent,population)
    # Randomly choose connection
    if len(available) > 0:
        connection = np.random.choice(available)
        # Make connection
        population.iloc[connection].connections.append(agent)
        population.iloc[agent].connections.append(connection)

        # Update number of connections
        population['num_connections'] = population.connections.apply(len)
        if _cnt < 10:
            while population.num_connections[agent] < num_connections:
                _build_connection_list(agent,
                                       population,
                                       num_connections,
                                       _cnt=_cnt)

In [4]:
def run_experiment(num_connections=None,base=None,start=1,stop=10):
    _data = {}
    for i in range(start,stop):
        num_people = i*base
        _start = time.time()
        _ = ConnectionEngine(num_people=num_people,num_connections=num_connections)
        _.create_connections()
        _end = time.time()
        _data[num_people] = {
            'num_connections' :num_connections,
            'time': _end-_start
        }
    
    return _data

In [ ]:
%%time
connections = ConnectionEngine(num_people=100000,num_connections=50)

In [ ]:
%%time
connections.create_connections()

In [ ]:
connections.population.head(10)

In [ ]:
%%time
data = run_experiment(num_connections=3,base=100,start=1,stop=5)

In [ ]:
!touch experiment.py

In [ ]:
_start = time.time()
_.create_connections()
_end = time.time()
print(float('{:.2f}'.format(_end-_start)))

In [ ]:
_.population